# Capstone - Delivery-3
This dataset is part of the [Farming Systems Project](https://www.ars.usda.gov/northeast-area/beltsville-md-barc/beltsville-agricultural-research-center/sustainable-agricultural-systems-laboratory/docs/farming-systems-project/) at USDA, Beltsville MD.  This data is not available online on the USDA
 website but can be found on my [GitHub](https://github.com/mmtokay/DATA606/tree/master/datasets).


The data is split in two files, one that contains crop information and other with weather data.

Crop file:
* Crop - wheat, corn or soybean           
* GrowingSeason - year crop was cultivated 
* SystemName - crop management (traditional: NT and CT; organic: Org2, Org3 and Org6')    
* GrainYield - grain yield measured in kg/ha     
* PlantingDate - date seeds were planted  
* HarvestDate - date crop was harvested


Weather file:
* Year 
* Julian Day 
* Month
* Day
* Date
* avgtTempC - average temperature in C
* maxTempC - maximum temperature in C
* minTempC - minimum temperature in C
* maxHumPct - maximum humidity in %
* minHumPct - minimum humidity in %
* avgRadWm-2 - average radiation in w/m2
* meanWindMs-1 - mean wind in m/s
* PrecipitationMm - precipitation/snow melt in mm

# Feature Engineering


In [1]:
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import warnings
import time

from datetime import datetime, timedelta
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, RobustScaler, Normalizer, MinMaxScaler, StandardScaler, Binarizer
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.utils import shuffle
from time import time
from google.colab import drive
drive.mount('/content/drive')

warnings.simplefilter(action='ignore', category=FutureWarning)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Crop Data
Import crop data file.

In [2]:
data = pd.read_csv('/content/drive/My Drive/DATA606/Delivery-3/FSPGrainYieldsV3Clean.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113 entries, 0 to 1112
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Crop           1113 non-null   object 
 1   GrowingSeason  1113 non-null   int64  
 2   SystemName     1113 non-null   object 
 3   GrainYield     1113 non-null   float64
 4   PlantingDate   1113 non-null   object 
 5   HarvestDate    1042 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 52.3+ KB


In [3]:
# Convert from object to float64
data['GrainYield'] = pd.to_numeric(data.GrainYield, errors='coerce')

# Convert PlantingDate and HarvestDate from object to date
data['PlantingDate'] = pd.to_datetime(data.PlantingDate)
data['HarvestDate'] = pd.to_datetime(data.HarvestDate)

print("\nLet's check if there is any data missing on the dataset.\n")
data.isna().sum()


Let's check if there is any data missing on the dataset.



Crop              0
GrowingSeason     0
SystemName        0
GrainYield        0
PlantingDate      0
HarvestDate      71
dtype: int64

In [4]:
# Calculate duration between PlantingDate and HarvestDate
data['weekDuration'] = data['HarvestDate'] - data['PlantingDate']
data['weekDuration'] = data['weekDuration']/np.timedelta64(1,'W')
print('\nCheck unique values for Crop, GrowingSeason and SystemName columns.\n')
print("Crop", data.Crop.unique())
print("\nGrowing Season", data.GrowingSeason.unique())
print("\nCrop Management Type", data.SystemName.unique())
print("\nThere are duplicate values for SystemManagement because column values are case sensitive. I will convert SystemName column to uppercase.")
data['SystemName'] = data['SystemName'].str.upper()
print("\nCrop Management Type", data.SystemName.unique())


Check unique values for Crop, GrowingSeason and SystemName columns.

Crop ['CRN' 'SOY' 'WHT']

Growing Season [1996 1997 1998 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010
 2011 2012 2013 2014 2015 2016]

Crop Management Type ['NT' 'CT' 'Org2' 'Org3' 'Org6' 'ORG2' 'ORG3' 'ORG6']

There are duplicate values for SystemManagement because column values are case sensitive. I will convert SystemName column to uppercase.

Crop Management Type ['NT' 'CT' 'ORG2' 'ORG3' 'ORG6']


In [5]:
# 1 for conventional
# 0 for organic
data['SystemNameType'] = ((data.SystemName == "NT") | (data.SystemName == "CT")).map({True:'1', False:'0'})
# Drop SystemName column
data.drop('SystemName', axis=1, inplace=True)
data.head()

,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
0,CRN,1996,10488.0,1996-05-23,1996-10-17,21.0,1
1,CRN,1996,9722.5,1996-05-23,1996-10-17,21.0,1
2,CRN,1996,10008.0,1996-05-23,1996-10-17,21.0,1
3,CRN,1996,8928.8,1996-05-23,1996-10-17,21.0,1
4,CRN,1996,10985.3,1996-05-23,1996-10-17,21.0,1


I will separate the data by crop: corn, soybean and wheat and I will display basic statistics for each crop.

# Corn dataset - Statistics

In [6]:
data_corn = data.loc[data['Crop'] == "CRN"]
data_corn.reset_index(inplace = True)
data_corn.describe(include="all")

,index,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
count,390.000000,390,390.000000,390.000000,390,370,370.000000,390
unique,NaN,1,NaN,NaN,43,64,NaN,2
top,NaN,CRN,NaN,NaN,2009-06-29 00:00:00,2008-10-10 00:00:00,NaN,0
freq,NaN,390,NaN,NaN,16,20,NaN,234
first,NaN,NaN,NaN,NaN,1996-05-23 00:00:00,1996-10-17 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2016-06-09 00:00:00,2016-10-18 00:00:00,NaN,NaN
mean,194.500000,NaN,2006.376923,5746.695385,NaN,NaN,21.583398,NaN
std,112.727548,NaN,5.932697,3350.745309,NaN,NaN,2.521325,NaN
min,0.000000,NaN,1996.000000,71.900000,NaN,NaN,16.857143,NaN
25%,97.250000,NaN,2002.000000,2811.000000,NaN,NaN,19.428571,NaN


# Soybean dataset - Statistics

In [7]:
data_soy = data.loc[data['Crop'] == "SOY"]
data_soy.reset_index(inplace = True)
data_soy.describe(include="all")

,index,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
count,500.000000,500,500.000000,500.000000,500,465,465.000000,500
unique,NaN,1,NaN,NaN,49,49,NaN,2
top,NaN,SOY,NaN,NaN,2001-05-30 00:00:00,2013-10-29 00:00:00,NaN,1
freq,NaN,500,NaN,NaN,20,20,NaN,273
first,NaN,NaN,NaN,NaN,1996-05-23 00:00:00,1996-10-04 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2016-07-14 00:00:00,2016-10-27 00:00:00,NaN,NaN
mean,639.500000,NaN,2006.768000,2647.311400,NaN,NaN,21.113978,NaN
std,144.481833,NaN,5.767556,1156.686365,NaN,NaN,2.868903,NaN
min,390.000000,NaN,1996.000000,165.200000,NaN,NaN,15.000000,NaN
25%,514.750000,NaN,2002.000000,1888.100000,NaN,NaN,19.142857,NaN


# Wheat dataset - Statistics

In [8]:
data_wheat = data.loc[data['Crop'] == "WHT"]
data_wheat.reset_index(inplace = True)
data_wheat.describe(include="all")

,index,Crop,GrowingSeason,GrainYield,PlantingDate,HarvestDate,weekDuration,SystemNameType
count,223.000000,223,223.000000,223.000000,223,207,207.000000,223
unique,NaN,1,NaN,NaN,19,15,NaN,2
top,NaN,WHT,NaN,NaN,2001-10-26 00:00:00,2015-07-12 00:00:00,NaN,1
freq,NaN,223,NaN,NaN,16,16,NaN,115
first,NaN,NaN,NaN,NaN,1996-11-04 00:00:00,1997-07-06 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,2015-11-17 00:00:00,2016-07-11 00:00:00,NaN,NaN
mean,1001.000000,NaN,2007.578475,4278.827354,NaN,NaN,35.621808,NaN
std,64.518731,NaN,6.094000,1064.753320,NaN,NaN,1.901822,NaN
min,890.000000,NaN,1997.000000,1403.500000,NaN,NaN,31.428571,NaN
25%,945.500000,NaN,2001.500000,3671.850000,NaN,NaN,34.857143,NaN


# Week Duration For Each Crop
I will use the minimum week duration to generate weather features:

corn = 16

soybeans = 15

wheat = 31


# Weather Data

Import weather data.

In [10]:
weather_data = pd.read_csv('/content/drive/My Drive/DATA606/Delivery-3/FSPWeather1996-2019V2.csv')
weather_data['Date'] = pd.to_datetime(weather_data.Date)
print(weather_data.describe(include="all"))
print("\nLet's check if there is any data missing on the dataset.\n")
print(weather_data.isna().sum())

               Year    JulianDay  ...  meanWindMs-1  PrecipitationMm
count   8763.000000  8763.000000  ...   8750.000000      8763.000000
unique          NaN          NaN  ...           NaN              NaN
top             NaN          NaN  ...           NaN              NaN
freq            NaN          NaN  ...           NaN              NaN
first           NaN          NaN  ...           NaN              NaN
last            NaN          NaN  ...           NaN              NaN
mean    2007.498916   183.092320  ...      1.999463         2.941006
std        6.921633   105.423565  ...      1.366765         8.284377
min     1996.000000     1.000000  ...      0.000000         0.000000
25%     2001.500000    92.000000  ...      1.128250         0.000000
50%     2007.000000   183.000000  ...      1.660000         0.000000
75%     2013.000000   274.000000  ...      2.479800         1.000000
max     2019.000000   366.000000  ...     28.050000       137.000000

[13 rows x 13 columns]

Let's che

In [11]:
weather_data.drop(['Year','JulianDay','Month',' Day','avgRadWm-2'], axis=1, inplace=True)
print(weather_data.describe(include="all"))

                       Date    avgtTempC  ...  meanWindMs-1  PrecipitationMm
count                  8763  8728.000000  ...   8750.000000      8763.000000
unique                 8763          NaN  ...           NaN              NaN
top     2007-09-10 00:00:00          NaN  ...           NaN              NaN
freq                      1          NaN  ...           NaN              NaN
first   1996-01-01 00:00:00          NaN  ...           NaN              NaN
last    2019-12-31 00:00:00          NaN  ...           NaN              NaN
mean                    NaN    13.059551  ...      1.999463         2.941006
std                     NaN     9.363676  ...      1.366765         8.284377
min                     NaN   -13.600000  ...      0.000000         0.000000
25%                     NaN     5.300000  ...      1.128250         0.000000
50%                     NaN    13.500000  ...      1.660000         0.000000
75%                     NaN    21.492350  ...      2.479800         1.000000

My next step is to work on feature engineering, I will combine crop mangement type in two categories traditional and organic.  After I determine the average number of weeks that takes for each crop to mature I will calculate weather variables weekly average starting from planting date.

I will also calculate growing degree days (GDD) that "are used to estimate the growth and development of plants and insects during the growing season. The basic concept is that development will only occur if the temperature exceeds some minimum development threshold, or base temperature (TBASE). The base temperatures are determined experimentally and are different for each organism". [1]

GDD formula for corn and soybean:

GDD = (Daily Max Temp °C + Daily Min Temp °C) / 2 - 10

GDD formula wheat:

GDD = (Daily Max Temp °C + Daily Min Temp °C) / 2 - 4.4

In [0]:
def calcGDD(df,startDate,endDate,factor):
    gdd = 0
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        gdd = gdd + (((j['maxTempC']+j['minTempC'])/2)-factor)
    return gdd

def calcAverage(df,startDate,endDate,var):
    sum = 0
    avg = 0
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        sum = sum + j[var]
    if sum > 0:
        avg = sum/(i+1)
    return avg

def calcMax(df,startDate,endDate,var):
    val = []
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        val.append(j[var])
    maxVal = max(val)
    return maxVal

def calcMin(df,startDate,endDate,var):
    val = []
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        val.append(j[var])
    minVal = min(val)
    return minVal

def calcSum(df,startDate,endDate,var):
    sum = 0
    for i, j in df.loc[(df.Date >= startDate) & (df.Date <= endDate)].iterrows():
        sum = sum + j[var]
    return sum

def createFeaturesMatrix(cropData,weatherData,numWeeks,GDDFactor):
    master_tp = list()
    colName = ()
    i = 0
    j = 0
    for i, j in cropData.iterrows():
        if (i == 0):
            startDate = j['PlantingDate']
        #start calculating date ranges to aggregate weather data for 16 weeks starting from plantingDate
        new_tp = ()
        for w in range(numWeeks):
            temp_tuple = ()
            beginWeek = j['PlantingDate'] + timedelta(days=7)*w
            endWeek = j['PlantingDate'] + timedelta(days=7)*(w+1)
            if(w==(numWeeks-1)):
                temp_tuple = (calcAverage(weather_data,beginWeek,endWeek,'avgtTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxTempC'),\
                              calcMin(weather_data,beginWeek,endWeek,'minTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxHumPct'),\
                              calcMin(weather_data,beginWeek,endWeek,'minHumPct'),\
                              calcAverage(weather_data,beginWeek,endWeek,'meanWindMs-1'),\
                              calcSum(weather_data,beginWeek,endWeek,'PrecipitationMm'),\
                              calcGDD(weather_data,startDate,endWeek,GDDFactor),\
                              j['SystemNameType'],j['GrainYield'])
                if (i == 0):
                    colName = colName + ('avgTemp'+str(w+1),'maxTemp'+str(w+1),'minTemp'+str(w+1),\
                                         'maxHum'+str(w+1),'minHum'+str(w+1),'meanWind'+str(w+1),\
                                         'Precip'+str(w+1),'GDD','SystemNameType','GrainYield')
            else:
                temp_tuple = (calcAverage(weather_data,beginWeek,endWeek,'avgtTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxTempC'),\
                              calcMin(weather_data,beginWeek,endWeek,'minTempC'),\
                              calcMax(weather_data,beginWeek,endWeek,'maxHumPct'),\
                              calcMin(weather_data,beginWeek,endWeek,'minHumPct'),\
                              calcAverage(weather_data,beginWeek,endWeek,'meanWindMs-1'),\
                              calcSum(weather_data,beginWeek,endWeek,'PrecipitationMm'))
                if (i == 0):
                    colName = colName + ('avgTemp'+str(w+1),'maxTemp'+str(w+1),'minTemp'+str(w+1),\
                                         'maxHum'+str(w+1),'minHum'+str(w+1),'meanWind'+str(w+1),\
                                         'Precip'+str(w+1))
            new_tp = new_tp + temp_tuple
        #print(new_tp)
        master_tp.append(new_tp)

    new_df = pd.DataFrame(list(master_tp),columns = colName)
    return(new_df)

# Corn

In [14]:
data_corn.drop(['Crop','GrowingSeason','HarvestDate','weekDuration'], axis=1, inplace=True)

KeyError: ignored

In [0]:
new_df16 = createFeaturesMatrix(data_corn,weather_data,16,10)
new_df16.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/cornFeatures16w.csv', index = False, header=True)

In [0]:
new_df15 = createFeaturesMatrix(data_corn,weather_data,15,10)
new_df15.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/cornFeatures15w.csv', index = False, header=True)

In [0]:
new_df14 = createFeaturesMatrix(data_corn,weather_data,14,10)
new_df14.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/cornFeatures14w.csv', index = False, header=True)

In [0]:
new_df13 = createFeaturesMatrix(data_corn,weather_data,13,10)
new_df13.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/cornFeatures13w.csv', index = False, header=True)

# Soybean

In [19]:
data_soy.drop(['Crop','GrowingSeason','HarvestDate','weekDuration'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
new_soy_df15 = createFeaturesMatrix(data_soy,weather_data,15,10)
new_soy_df15.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/soyFeatures15w.csv', index = False, header=True)

In [0]:
new_soy_df14 = createFeaturesMatrix(data_soy,weather_data,14,10)
new_soy_df14.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/soyFeatures14w.csv', index = False, header=True)

In [0]:
new_soy_df13 = createFeaturesMatrix(data_soy,weather_data,13,10)
new_soy_df13.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/soyFeatures13w.csv', index = False, header=True)

In [0]:
new_soy_df12 = createFeaturesMatrix(data_soy,weather_data,12,10)
new_soy_df12.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/soyFeatures12w.csv', index = False, header=True)

# Wheat

In [25]:
data_wheat.drop(['Crop','GrowingSeason','HarvestDate','weekDuration'], axis=1, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [0]:
new_wheat_df31 = createFeaturesMatrix(data_wheat,weather_data,31,4.4)
new_wheat_df31.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/wheatFeatures31w.csv', index = False, header=True)

In [0]:
new_wheat_df30 = createFeaturesMatrix(data_wheat,weather_data,30,4.4)
new_wheat_df30.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/wheatFeatures30w.csv', index = False, header=True)

In [0]:
new_wheat_df29 = createFeaturesMatrix(data_wheat,weather_data,29,4.4)
new_wheat_df29.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/wheatFeatures29w.csv', index = False, header=True)

In [0]:
new_wheat_df28 = createFeaturesMatrix(data_wheat,weather_data,28,4.4)
new_wheat_df28.to_csv(r'/content/drive/My Drive/DATA606/Delivery-3/wheatFeatures28w.csv', index = False, header=True)